In [12]:
# https://pieriantraining.com/tensorflow-lstm-example-a-beginners-guide/
# https://www.tensorflow.org/guide/keras/working_with_rnns

In [13]:
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
import keras
from keras.models import load_model
from keras import layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import joblib

In [14]:
dataset = pd.read_csv('../../data/preprocessed_data.csv',encoding='ISO-8859-1')
dataset = dataset.drop(['Unnamed: 0'], axis=1)
dataset = dataset.dropna(subset=['Text'])
dataset = dataset.sample(frac=0.1, random_state=42)
dataset

,Text,Source,Human
287452,ING AsiaPacific Companys Problems Research Pap...,Human,1
222199,Crisis Love Inquiry Essay Critical Writing fol...,Human,1
453600,Sure sex segregation makes lot sense many spor...,Human,1
276338,Christianity Islam Values Essay Christianity f...,Human,1
78217,Becca liked swim practiced everyday hours ente...,GLM-130B,0
...,...,...,...
663613,Mass Eoghan Chada 10 brother Ruairi 5 said St ...,OPT-30B,0
285976,Asian Teachers Polish Lesson Perfection Stigle...,Human,1
679335,Move knife slowly avoid slipping accidentally ...,OPT-6.7B,0
775773,Good dreams likely occur person feeling relaxe...,Text-Davinci-003,0


In [15]:
# Initialize the tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(dataset['Text'])

In [16]:
# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(dataset['Text'])

# Pad sequences to ensure uniform input length
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [17]:
# unitializing label encoder
label_encoder = LabelEncoder()
label_encoder.fit(dataset['Human'])

# labels to numerical format conversion
encoded_labels = label_encoder.transform(dataset['Human'])

In [33]:
# LSTM instantiation
lstm = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(64, return_sequences=False),
    Dense(32),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [34]:
lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [35]:
# training phase
lstm.fit(padded_sequences, encoded_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


1973/1973 ━━━━━━━━━━━━━━━━━━━━ 34s 16ms/step - accuracy: 0.7032 - loss: 0.5575 - val_accuracy: 0.7765 - val_loss: 0.4434
Epoch 2/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - accuracy: 0.7952 - loss: 0.4131 - val_accuracy: 0.8137 - val_loss: 0.3858
Epoch 3/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 30s 15ms/step - accuracy: 0.8308 - loss: 0.3547 - val_accuracy: 0.8178 - val_loss: 0.3844
Epoch 4/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 45s 23ms/step - accuracy: 0.8601 - loss: 0.3025 - val_accuracy: 0.8277 - val_loss: 0.3760
Epoch 5/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 45s 23ms/step - accuracy: 0.8810 - loss: 0.2677 - val_accuracy: 0.8312 - val_loss: 0.4072
Epoch 6/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 30s 15ms/step - accuracy: 0.8965 - loss: 0.2373 - val_accuracy: 0.8257 - val_loss: 0.4218
Epoch 7/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 30s 15ms/step - accuracy: 0.9212 - loss: 0.1911 - val_accuracy: 0.8272 - val_loss: 0.4683
Epoch 8/10
1973/1973 ━━━━━━━━━━━━━━━━━━━━ 30s 15ms/step - accuracy: 0.9394 - loss: 0.15

In [36]:
# 0.9616

In [37]:
lstm.save('../../models/NeuralNetworks/lstm_two_96.keras')